In [147]:
!pip install santorinai


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [148]:
import random
import ast
import numpy as np

In [149]:
from santorinai.tester import Tester
from santorinai.player_examples.random_player import RandomPlayer
from santorinai.player_examples.first_choice_player import FirstChoicePlayer

# Basic Rules

In [150]:
def movesPlayer(board, players, x, y):
    z = board[x][y]
    moves = []
    for i in range(-1,2):
        for j in range(-1,2):
            if i == 0 and j == 0:
                continue
            if x+i < 0 or x+i >= len(board):
                continue
            if y+j < 0 or y+j >= len(board):
                continue
            if board[x+i][y+j] in (z-2, z-1, z, z+1) and board[x+i][y+j] < 4 and (x+i, y+j) not in players:
                moves.append((x+i, y+j))
    return moves

In [151]:
""""
def movesPlayer(board, players, x, y):
    z = board[x][y]
    moves = []
    for i in range(-1,2):
        for j in range(-1,2):
            if i == 0 and j == 0:
                continue
            if x+i < 0 or x+i >= len(board):
                continue
            if y+j < 0 or y+j >= len(board):
                continue
            if board[x+i][y+j] in (z-1, z, z+1) and (x+i, y+j) not in players:
                moves.append((x+i, y+j))
    return moves
"""

'"\ndef movesPlayer(board, players, x, y):\n    z = board[x][y]\n    moves = []\n    for i in range(-1,2):\n        for j in range(-1,2):\n            if i == 0 and j == 0:\n                continue\n            if x+i < 0 or x+i >= len(board):\n                continue\n            if y+j < 0 or y+j >= len(board):\n                continue\n            if board[x+i][y+j] in (z-1, z, z+1) and (x+i, y+j) not in players:\n                moves.append((x+i, y+j))\n    return moves\n'

In [152]:
def constructsPlayer(board, players, x, y):
    places = []
    for i in range(-1,2):
        for j in range(-1,2):
            if i == 0 and j == 0:
                continue
            if x+i < 0 or x+i >= len(board):
                continue
            if y+j < 0 or y+j >= len(board):
                continue
            if board[x+i][y+j] < 4 and (x+i, y+j) not in players:
                places.append((x+i, y+j))   
    return places

In [153]:
def win(board, x, y):
    return board[x][y] == 3

# MonteCarlo Simulation

In [154]:
def randomWin(board, players, playerAct):
    for i in range(4):
        if win(board, *players[i]):
            return i
    for i in range(100):
        playerAct = (playerAct + 1) % 4
        moves = movesPlayer(board, players, *players[playerAct])
        if len(moves) > 0:
            players[playerAct] = moves[random.randint(0, len(moves)-1)]
            if win(board, *players[playerAct]):
                return playerAct
            constructs = constructsPlayer(board, players, *players[playerAct])
            if len(constructs) > 0:
                construct = constructs[random.randint(0, len(constructs)-1)]
                board[construct[0]][construct[1]] += 1
    return -1

In [155]:
def simulate(nbGames, myboard, myplayers, myplayerAct, maximizingPlayer):
    nbWin = 0
    for i in range(nbGames):
        board = myboard[:]
        players = myplayers[:]
        rd = randomWin([row[:] for row in board], players[:], myplayerAct)
        if rd == -1:
            nbGames -= 1
        else:
            nbWin += 1 if rd in (myplayerAct,(myplayerAct+2)%4) else 0
    if nbGames == 0:
        return 0
    return nbWin/nbGames * 2 - 1 if maximizingPlayer else 1 - nbWin/nbGames * 2

# Minimax

In [156]:
#do a minimax search to find the best move

def minimax(board, players, playerAct, depth, alpha, beta, maximizingPlayer, n):
    if win(board, *players[(playerAct-1)%4]):
        return -1 if maximizingPlayer else 1
    if depth == 0:
        return simulate(n, board, players, playerAct, maximizingPlayer)
    isBreak = False
    for move in movesPlayer(board, players, *players[playerAct]):
        if isBreak:
            break
        newPlayers = players[:]
        newPlayers[playerAct] = move
        for construct in constructsPlayer(board, newPlayers, *newPlayers[playerAct]):
            newBoard = [row[:] for row in board]
            newBoard[construct[0]][construct[1]] += 1
            value = minimax(newBoard, newPlayers, (playerAct+1)%4, depth-1, alpha, beta, not maximizingPlayer, n)
            if maximizingPlayer:
                alpha = max(alpha, value)
                if alpha >= 1:
                    isBreak = True
                    break
            else:
                beta = min(beta, value)
                if beta <= -1:
                    isBreak = True
                    break
            if beta <= alpha:
                isBreak = True
                break
    return alpha if maximizingPlayer else beta


In [157]:
def getBestMove(board, players, playerAct, depth, n = 100):
    bestValue = -1
    bestMove = None
    bestConstruct = None
    while bestMove == None:
        if depth == 0:
            return movesPlayer(board, players, *players[playerAct])[0], constructsPlayer(board, players, *players[playerAct])[0], 0, 0
        for move in movesPlayer(board, players, *players[playerAct]):
            newPlayers = players[:]
            newPlayers[playerAct] = move
            for construct in constructsPlayer(board, newPlayers, *newPlayers[playerAct]):
                newBoard = [row[:] for row in board]
                newBoard[construct[0]][construct[1]] += 1
                value = minimax(newBoard, newPlayers, (playerAct+1)%4, depth-1, bestValue, 1, False, n)
                if value > bestValue:
                    bestValue = value
                    bestMove = move
                    bestConstruct = construct
                if bestValue >= 1:
                    return bestMove, bestConstruct, bestValue
        depth -= 1
    return bestMove, bestConstruct, bestValue

In [158]:
from santorinai.player import Player

class MyPlayer(Player):
    """
    My player description
    """

    def name(self):
        return "Tkt bot"

    # Placement of the pawns
    def place_pawn(self, board, pawn):
        my_choice = (2,2) if (2,2) in board.get_possible_movement_positions(pawn) else random.choice(board.get_possible_movement_positions(pawn))
        return my_choice

    # Movement and building
    def play_move(self, board, pawn):
        pos, construct, i = getBestMove(board.board, [player.pos for player in  board.pawns], pawn.number -1, 2, 500)
        return pos, construct

In [159]:
from santorinai.player import Player

class MyPlayer2(Player):
    """
    My player description
    """

    def name(self):
        return "Tkt bot 2"

    # Placement of the pawns
    def place_pawn(self, board, pawn):
        my_choice = (2,2) if (2,2) in board.get_possible_movement_positions(pawn) else random.choice(board.get_possible_movement_positions(pawn))
        return my_choice

    # Movement and building
    def play_move(self, board, pawn):
        pos, construct, i = getBestMove(board.board, [player.pos for player in  board.pawns], pawn.number -1, 3, 100)
        return pos, construct

In [165]:
tester = Tester()
tester.delay_between_moves = 0
tester.display_board = False

p1 = MyPlayer2()
p2 = FirstChoicePlayer()

tester.play_1v1(p1, p2, nb_games = 5)

Game 1
Player Firsty First is placing pawn 1
   Pawn placed at position (0, 0)
Player Tkt bot 2 is placing pawn 2
   Pawn placed at position (2, 2)
Player Firsty First is placing pawn 3
   Pawn placed at position (0, 1)
Player Tkt bot 2 is placing pawn 4
   Pawn placed at position (1, 0)

Playing the game
   Current pawn: pawn 1 of player 1 at position (0, 0)
Player Tkt bot 2 is moving pawn 1
   Pawn moved at position (1, 1) and built at position (0, 0)

_0 _0 _0 _0 _0 
_0 _0 _0 _0 _0 
_0 _0 20 _0 _0 
30 10 _0 _0 _0 
_1 40 _0 _0 _0 

   Current pawn: pawn 2 of player 2 at position (2, 2)
Player Firsty First is moving pawn 2
   Pawn moved at position (1, 2) and built at position (0, 2)

_0 _0 _0 _0 _0 
_0 _0 _0 _0 _0 
_1 20 _0 _0 _0 
30 10 _0 _0 _0 
_1 40 _0 _0 _0 

   Current pawn: pawn 3 of player 1 at position (0, 1)
Player Tkt bot 2 is moving pawn 3
   Pawn moved at position (0, 2) and built at position (0, 1)

_0 _0 _0 _0 _0 
_0 _0 _0 _0 _0 
31 20 _0 _0 _0 
_1 10 _0 _0 _0 
_1 40 _0

In [ ]:
board = [[0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0]]

players = [(0,0), (4,4), (0,4), (4,0)]
print(getBestMove(board, players, 0, 2, 100))

((1, 1), (0, 1), -0.09999999999999998)


In [ ]:
board = [[0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0]]

players = [(0,0), (4,4), (0,4), (4,0)]
print(getBestMove(board, players, 0, 2, 500))

((0, 1), (0, 2), -0.05858585858585863)


In [ ]:
board = [[0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0]]

players = [(0,0), (4,4), (0,4), (4,0)]
print(getBestMove(board, players, 0, 3, 100))

((0, 1), (0, 2), 0.16000000000000003)
